# Translate docs into other languages

In [1]:
from googletrans import Translator

import os
from os import walk
from os import listdir
from os.path import isfile, join
import time

import openai

## ChatGPT API

In [2]:
openai.api_key = ''


def send_message(prompt, message):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "user",
                "content": prompt + message,
            },
        ],
    )
    return completion.choices[0].message.content

## Google translate API

In [3]:
translator = Translator()
translated = translator.translate('Mitä sinä teet')
print(translated.text)

What are you doing


## Translation

### Folders

In [4]:
Folder_list = ['../en/Dessert','../en/Main_Course']

Post_list = []

In [5]:
for folder in Folder_list:
    for root, directories, files in os.walk(folder, topdown=False):
        for name in files:
            if '.md' in name:
                Post_list.append(os.path.join(root, name))
Post_list[0]

'../en/Dessert/Hardware/2021-06-03-Mac-mini.md'

In [6]:
len(Post_list)

95

In [10]:
vincent_docs = []
for path_ in Post_list:
    with open(path_, 'r') as f:
        if 'Vincent' in ''.join(f.readlines()):
            vincent_docs.append(path_)

In [11]:
len(vincent_docs)

40

Move the docs to an new folder

In [13]:
for file_ in vincent_docs:
    !cp $file_ /Users/vincent.yuan/Downloads/vincent_docs

### Google translation processing func

In [ ]:
def translate_post(text_lines, translator, lang):
    split_point_ = 0
    # Find the position that starts the body, the headers and body should be processed separately
    for no_, block_ in enumerate(text_lines):
        if '##' in block_:
            split_point_ = no_
            print(f'Header ends at no.{split_point_} block...')
            break
    
    
    headers_ = text_lines[:split_point_]
    body_ = text_lines[split_point_:]
    
    def translate_header_(_headers_, lang):
        if lang == 'en':
            en_headers_ = _headers_.replace('作者', 'Author:')\
                                    .replace('发布于', 'Posted on ')\
                                    .replace('阅读时间：约','Reading time: ')\
                                    .replace('分钟', ' mins')\
                                    .replace('微信公众号文章链接','WeChat Post Link:')
            return en_headers_
        else:
            print(f'{lang} cannot be well verified, so the original headers are returned.')
            return _headers_
    
    def translate_body_(body_: list, lang: str):
        import time
        import re
        en_body_ = []
        if lang == 'en':
            for text_ in body_:
                # exclusion of special chars
                if 'img' in text_ or text_ == '\n':
                    en_body_.append(text_)
                
                # Skip hyper links
                elif re.findall(r'(\[.+\]\(.+\))', text_):
                    
                    hyper_link_ = re.findall(r'(\[.+\]\(.+\))', text_)

                    if hyper_link_:

                        split_text = re.split(r'(\[.+\]\(.+\))', text_, maxsplit=0)
                        for txt_ in split_text:
                            
                            if txt_ != hyper_link_ and txt_ != '' and  txt_ != '\n':
                                try:
                                    en_body_.append(translator.translate(str(txt_)).text)
                                except:
                                    print(txt_)
                                    pass
                            
                            elif  txt_ != '' and  txt_ != '\n':
                                en_body_.append(txt_)
                            else:
                                pass
                else:
                    # Sleep 300 milliseconds
                    time.sleep(300/1000)
                    en_body_.append(translator.translate(str(text_)).text)
            return en_body_
        else:
            print(f'{lang} cannot be well verified, so the original body are returned.')
            return body_
    
    translated_headers = translate_header_(''.join(headers_), lang)
    translated_body = translate_body_(body_, lang)
    
    return translated_headers + '\n'.join(translated_body)
    

### Translation in process

In [ ]:

prompt = "translate below text into English: "


undone_post_key = []
chn_posts = {}
en_posts = {}
for post_path in Post_list:
    print(f'Processing {post_path}...')
    with open(post_path, 'r') as f:
        post_text_ = f.readlines()
        # if 'Vincent' in ''.join(post_text_):
        chn_posts[post_path] = post_text_
        # split the text if the number of tokens are greater than 4097
        # try:
        #     en_posts[post_path] = send_message(prompt=prompt,
        #                                         message=''.join(post_text_)
        #                                         )
        # except:
        #     undone_post_key.append(post_path)




### Append post headers

In [ ]:
test_text = """

---
template: overrides/blogs.html
tags:
  - python
---


# test



"""

In [ ]:
test_text_copy = test_text.lstrip('\n').split('---')
test_text_copy

In [ ]:
def find_post_headers(post_text):
    header = post_text.lstrip('\n').split('---')[1]
    header = "---" + header + '---\n\n'
    return header


In [ ]:
for path_, text_ in chn_posts.items():
    print(path_)
    print(find_post_headers(post_text=''.join(text_)))
    break

### EN post heahder

In [ ]:
en_header_copy = chn_posts.copy()
en_header_dict = {}
for path, text in en_header_copy.items():
    en_path = path.replace('../','../en/')
    en_header_dict[en_path] = find_post_headers(post_text=''.join(text))

In [ ]:
test_key_ = list(en_header_dict.keys())[25]
print(test_key_)
print(en_header_dict[test_key_])

### EN post content

In [ ]:
en_posts_new = {}
for path, text in en_posts_copy.items():
    en_path = path.replace('../','../en/')
    en_posts_new[en_path] = text

In [ ]:
test_key = list(en_posts_copy.keys())[2]
print(en_posts_copy[test_key])

## Write the EN files

### Write headers

In [ ]:
from pathlib import Path

for en_path_, header_ in en_header_dict.items():
    filename = Path(en_path_).parent
    filename.mkdir(parents=True, exist_ok=True)
    filename.touch(exist_ok=True)
    try:
        with open(en_path_, 'r') as f:
            original_text_ = f.readlines()
            new_post_ = header_ + ''.join(original_text_)

            with open(en_path_, 'w') as new_file:
                new_file.write(new_post_)
    except:
        pass

### Write post

In [ ]:
from pathlib import Path

for en_path_, text_ in en_posts_new.items():
    filename = Path(en_path_).parent
    filename.mkdir(parents=True, exist_ok=True)
    filename.touch(exist_ok=True)
    with open(en_path_, 'w+') as f:

        f.writelines(text_)